# Data Preparation

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Evaluation

In [12]:
# import package for displaying NER results
from spacy import load, displacy
import pandas as pd

In [13]:
# import model
link_to_model = "./output/model-last"
loaded_model = load(link_to_model)

/Users/syubbanfakhriya/Desktop/Repository/news_classification/venv/lib/python3.9/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'id_pipeline' (0.0.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [14]:
test_text = """beliau pulang dinas aceh melewati jalan lokasi penembakan kabid humas bandung polda maluku kombes Muhammad Roem ohoirat roem bharaka tewas ditembak jalan mengaku mengkonfirmasi bharaka ditembak berkendara jalan ditembak situlah versi situ berboncengan versi roem roem ditangani pom ad penjelasan terkait kronologi penembakan memeriksa pom ad diberitakan menembak bharaka pratu ri menembak rekannya anggota tni prada prada dilaporkan kritis meninggal kondisi kritis rumah sakit di aceh barat meninggal anggota brimob kapendam xvi pattimura kolonel arh adi prayogo choirul fajar wartawan rabu 163 penembakan anggota tni polri rabu 163 pratu ri menjalani pemeriksaan intensif polisi militer motif penembakan pratu ri di jakarta barat diduga mengalami depresi bandar pasir mandoge indikasinya kondisi depresi akut pemeriksaan psikologis barat kejiwaan didalami diperiksa"""

In [16]:
doc = loaded_model(test_text)
displacy.render(doc, style="ent", jupyter=True)

In [17]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

aceh 20 24 ORANG
bandung 70 77 LOKASI
maluku 84 90 LOKASI
aceh barat 480 490 LOKASI
fajar 571 576 ORANG
jakarta barat 717 730 LOKASI


In [19]:
df_test = pd.read_csv('./dataset/df_test_lower.csv')
df_test.head()

,title,date,description,source
0,"sempat tertutup longsor, akses jalan citengah ...","Rabu, 20 Apr 2022 11:22 WIB","ruas jalan citengah di desa baginda, kecamatan...",www.detik.com
1,"hujan deras, tebing tol pandaan arah malang lo...","Selasa, 08 Mar 2022 16:24 WIB",tebing di tol pandaan-malang km 77 hingga km 7...,www.detik.com
2,tebing 10 meter longsor timpa rumah di trenggalek,"Minggu, 27 Mar 2022 19:42 WIB",longsor kembali terjadi di trenggalek. kali in...,www.detik.com
3,luhut klaim tak bahas politik dengan ganjar: m...,"Kamis, 09 Jun 2022 13:30 WIB",menko kemaritiman dan investasi luhut binsar p...,www.detik.com
4,"gerai atm di bekasi terbakar, diduga karena ko...","Rabu, 27 Apr 2022 10:22 WIB",sebuah ruangan atm milik salah satu bank bumn ...,www.detik.com


In [21]:
lokasi_csv = './dataset/location.csv'
df_lokasi = pd.read_csv(lokasi_csv)
df_lokasi.head()

,nama,lokasi
0,bali,provinsi
1,bangka belitung,provinsi
2,banten,provinsi
3,bengkulu,provinsi
4,di yogyakarta,provinsi


In [22]:
flattened_lokasi = []
for i in range(0, df_lokasi.shape[0]):
  flattened_lokasi.append(df_lokasi.iloc[i, 0])

text ='jakarta timur'
if( text in flattened_lokasi):
  print('masuk')
else:
  print('enggak')

masuk


In [23]:
for i in range(0, 1):
    news = df_test.iloc[0, -2]
    print(news)
    doc = loaded_model(news)
    # print(doc.ent)
    ents = []
    visited = []
    for ent in doc.ents:
        print(ent.text, ent.label_)
        if ((ent.text in flattened_lokasi) and (ent.text not in visited)):
            ents.append(ent.text)
            visited.append(ent.text)

    print(ents)


ruas jalan citengah di desa baginda, kecamatan sumedang selatan yang sebelumnya tertimbun longsor kini sudah dapat dilalui kendaraan. permukaan jalannya pun sudah tampak bersih dari material longsoran. dari pantauan di lokasi, sejumlah kendaraan baik sepeda motor dan mobil pun sudah tampak hilir mudik di lokasi yang sebelumnya tertimbun longsor. salah satu bongkahan akar bambu yang sempat menutupi badan jalan pun kini sudah berada di pinggir jalan. petugas bpbd sumedang hedi hidaya mengatakan, akses jalan citengah sudah dapat diakses sejak pagi atau sekitar pukul 07.00 wib. "material tanah dan bongkahan akar bambu berhasil dievakuasi oleh warga secara bergotong royong sehingga akses jalan pun terbuka kembali," ungkapnya kepada detikjabar, rabu (20/4/2022). hedi memaparkan proses evakuasi material longsoran sendiri tidak menggunakan alat berat. akan tetapi semua dikerjakan oleh warga secara bergotong royong. "semua dibersihkan warga secara gotong royong," ucapnya. sebelumnya, kata hedi,

In [25]:
document_results = []
for i in range(0, df_test.shape[0]):
      news = df_test.iloc[i, -2]
      doc = loaded_model(news)

      lokasi = []
      tanggal = []
      orang = []
      visited = []
      for ent in doc.ents:
            if(ent.label_== 'LOKASI'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        if ((teks in flattened_lokasi) and (teks not in visited)):
                              lokasi.append(teks)
                              visited.append(teks)
                  else:
                        if ((ent.text in flattened_lokasi) and (ent.text not in visited)):
                              lokasi.append(ent.text)
                              visited.append(ent.text)
            elif(ent.label_== 'TANGGAL'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        if (teks not in visited):
                              tanggal.append(teks)
                              visited.append(teks)
                  else:
                        if (ent.text not in visited):
                              tanggal.append(ent.text)
                              visited.append(ent.text)
            elif(ent.label_== 'ORANG'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        if (teks not in visited):
                              orang.append(teks)
                              visited.append(teks)
                  else:
                        if (ent.text not in visited):
                              orang.append(ent.text)
                              visited.append(ent.text)

      visited.clear()

      if(len(orang)==0):
            trueTopik=0
      else:
            trueTopik=1
      if(len(tanggal)==0):
            trueTanggal=0
      else:
            trueTanggal=1
      if(len(lokasi)==0):
            trueLokasi=0
      else:
            trueLokasi=1

      document_results.append([
      orang,
      trueTopik,
      tanggal,
      trueTanggal,
      df_test.iloc[i,1],
      lokasi,
      trueLokasi,
      '',
      '',
      '',
      news,
      ])

writer = pd.DataFrame(document_results, columns=['Who','True Who','When','True When','date','Where','True Where', 'True Positive', 'True Negative', 'Relevansi Berita', 'Skimming News'])
writer.to_excel('./result/result_modelKecBenKrim.xlsx', index=False)